# Analysis Notebook

This notebook is used to analyze datasets using trained models as well as other imported segmentations. 



## Table of Contents

[1. Environment Initialization and Setup](#1-environment-initialization-and-setup)  
- [1.1 Configurations](#11-configurations)  

[2. Loading Datasets](#2-loading-datasets)  

[3. Inference and Visualization](#3-inference-and-visualization)  
- [3.1 Load Models](#31-load-models)  
- [3.2 Save Mask Overlays](#32-save-mask-overlays)  
  - [3.2a Dataset Masks](#32a-dataset-masks)  
  - [3.2b Model Prediction Masks](#32b-model-prediction-masks)  
- [3.3 IoU Heatmaps](#33-iou-heatmaps)  

[4. Evaluation](#4-evaluation)  
- [4.1 Comparing IoU Distributions of multiple models](#41-comparing-iou-distributions-of-multiple-models)  
- [4.2 Postprocessing/Results](#42-postprocessingresults)  

# 1. Environment Initialization and Setup

This cell prepares the runtime environment and project structure for evaluating our models:

- **GPU Selection and Logging**
    - `os.environ["CUDA_VISIBLE_DEVICES]`: choose which GPU to use for this notebook (here, GPU 3). This allows us to run this notebook while other notebooks (such as `SAGE_train.ipynb`) run on other GPUs.
    - `os.environ["TF_CPP_MIN_LOG_LEVEL"]`: suppress TensorFlow info/warning messages
    - `warnings.filterwarnings("ignore", message=".Model.state_updates.*")`: suppresses warnings related to state updates for cleaner output

It also sets various directories that need to be accessed throughout the notebook, as well as handles necessary package imports



In [ ]:
import os
import warnings
import sys

# Set which GPU to use dynamically (e.g., "4" for GPU 4)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# Suppress tensorflow WARNING and INFO logs, shows only ERRORs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" 

# Supress model state warnings

warnings.filterwarnings("ignore", message=".*Model.state_updates.*")

import tensorflow as tf
# Uncomment for debugging/TF build
# print("TensorFlow version:", tf.__version__)
# print("Built with CUDA:", tf.test.is_built_with_cuda())
# print("GPU Available:", tf.config.list_physical_devices('GPU'))


#==============================================================
# CORE Libraries
#==============================================================
#TODO: cleanup unused imports that are now in other .py files, or make a single __init__ file
import random, math, re, time, csv, tempfile
import numpy as np
import pandas as pd
import cv2
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

from tqdm.notebook import tqdm
from IPython.display import clear_output, display

#Fractal Dimension analyzer
import StereoFractAnalyzer as SF


#============================================================
# Project Directory Setup

# Root directory of the project
ROOT_DIR = os.path.abspath("../SAGE")
print("ROOT DIR:", ROOT_DIR)

# Subdirectories

PRETRAIN_DIR = os.path.join(ROOT_DIR,"pretrained_models") #Pretrained models folder, where models already trained are stored
MODEL_DIR = os.path.join(ROOT_DIR, "logs") # Directory to save logs and trained models
Results_DIR = os.path.join(ROOT_DIR, "Results") #directory to save morphology results, performance metrics, and visualizations
#TODO: remove for final version
DATA_DIR = os.path.abspath(os.path.join(ROOT_DIR, "../../Data/logs")) # Using this to save models into for storage on disk 


print("Pretrained models dir:", PRETRAIN_DIR)
print("MODEL_DIR: ", MODEL_DIR)
print("Results DIR:", Results_DIR)
print("DATA DIRECTORY:", DATA_DIR) #TODO: remove for final push

# ================================================
# Mask RCNN Setup
# ================================================


sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils, visualize
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.utils import print_verbose

mrcnn_dir = os.path.dirname(modellib.__file__)
model_file_path = os.path.join(mrcnn_dir,'model.py')
print("mrcnn directory:",mrcnn_dir)
print("Path to model.py:", model_file_path)

#Ensure COCO weights are available
COCO_MODEL_PATH = os.path.join(PRETRAIN_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

#TODO: use similar method for downloading pretrained models/images as COCO?

%matplotlib inline 


## 1.1 Configurations

Here, we create a subclass of the base Config class for our SAGE model. We can specify how many GPUs to train on (1 in this case, due to hardware limitations), images per gpu, as well as how many classes (including background class). 

In [ ]:
class SAGEConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "SAGE"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1 #+ 1# background + particle + cluster

  # Input image resizing
    # Generally, use the "square" resizing mode for training and predicting
    # and it should work well in most cases. In this mode, images are scaled
    # up such that the small side is = IMAGE_MIN_DIM, but ensuring that the
    # scaling doesn't make the long side > IMAGE_MAX_DIM. Then the image is
    # padded with zeros to make it a square so multiple images can be put
    # in one batch.
    # Available resizing modes:
    # none:   No resizing or padding. Return the image unchanged.
    # square: Resize and pad with zeros to get a square image
    #         of size [max_dim, max_dim].
    # pad64:  Pads width and height with zeros to make them multiples of 64.
    #         If IMAGE_MIN_DIM or IMAGE_MIN_SCALE are not None, then it scales
    #         up before padding. IMAGE_MAX_DIM is ignored in this mode.
    #         The multiple of 64 is needed to ensure smooth scaling of feature
    #         maps up and down the 6 levels of the FPN pyramid (2**6=64).
    # crop:   Picks random crops from the image. First, scales the image based
    #         on IMAGE_MIN_DIM and IMAGE_MIN_SCALE, then picks a random crop of
    #         size IMAGE_MIN_DIM x IMAGE_MIN_DIM. Can be used in training only.
    #         IMAGE_MAX_DIM is not used in this mode.
    IMAGE_RESIZE_MODE = "square"
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024
    # Minimum scaling ratio. Checked after MIN_IMAGE_DIM and can force further
    # up scaling. For example, if set to 2 then images are scaled up to double
    # the width and height, or more, even if MIN_IMAGE_DIM doesn't require it.
    # However, in 'square' mode, it can be overruled by IMAGE_MAX_DIM.
    IMAGE_MIN_SCALE = 0
    # Number of color channels per image. RGB = 3, grayscale = 1, RGB-D = 4
    # Changing this requires other changes in the code. See the WIKI for more
    # details: https://github.com/matterport/Mask_RCNN/wiki
    IMAGE_CHANNEL_COUNT = 3 #images are grayscale(8bit) so may need to change to 1
    
    MEAN_PIXEL = np.array([123.7, 116.8, 103.9]) #may need to change to one value for grayscale

    # Default 
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 128

    DETECTION_MAX_INSTANCES = 100
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH= 76# 76 for 200

    #non-maximum suppression threshold for detection
    DETECTION_MIN_CONFIDENCE = 0.6
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 25
    
    #EARLY STOPPING
    EARLY_STOPPING_MONITOR = 'val_loss'
    EARLY_STOPPING_PATIENCE = 20 #number of epochs with no improvement required to stop
    ES_RESTORE_BEST_WEIGHTS =True
    ES_MODE= "min"
    ES_VERBOSE = 0
    
config = SAGEConfig()
config.display()

# 2. Loading Datasets

Here we will load the TEM images that we want to analyze using a model. 

To compare model performance with other segmentation methods, we can also load any dataset that contains the images and masks created with that method (in the same format as the images and masks used for training)

Multiple datasets can be loaded into the `datasets` dictionary for easy access or comparison.

As in the training notebook, they are loaded using the `SAGE_Dataset` class

Define datasets to load in a list of tuples, containing:
1. The dataset name (i.e., `PROCI_Test`)
2. A boolean `use_results` indicating whether or not a results directory should be created.

In this example, we load two datasets:
* `PROCI_Test`: Real TEM images of soot that have been manually segmented by various analysts, used to determine the performance of our models and other segmentation methods.
* `PROCI_EDMWS`: The same TEM images as in `PROCI_Test`, but with masks created using an EDM-WS Method from [Sipkens' atems MATLAB tool](https://github.com/tsipkens/atems). We will treat this dataset as "predictions" to check against the `PROCI_Test` "ground truths".

As we are using these datasets for analysis purposes, set `use_results=True` to create corresponding results directories. 


        

In [ ]:
#List of tuples: (dataset_name, use_results)


datasets_to_load= [
    ('PROCI_Test', True),
    ('PROCI_EDMWS', True),
   
]
datasets = {}


for idx, (name, use_results) in enumerate(datasets_to_load, start=1):
    clear_output(wait=True)
    pbar_datasets = tqdm(total=len(datasets_to_load), desc="Loading datasets", dynamic_ncols=True, position=0, leave=True, initial=idx-1,bar_format="{l_bar}{bar} {n_fmt}/{total_fmt}")

    datasets[name] = utils.load_and_register_dataset(name,ROOT_DIR, Results_DIR, create_dirs=use_results)
   
    pbar_datasets.update(1)
   


Next, as in the training notebook, we will verify that the masks are correctly assigned for each dataset using `visualize.display_top_masks`.

In [ ]:
# Load and display random samples
#image_ids = np.random.choice(dataset_train.image_ids, 4)

for name, dataset in datasets.items():
    print(f"\n--- Dataset: {name} ---")
    if len(dataset.image_ids) ==0: 
        print("No images loaded")
        continue
    image_id = dataset.image_ids[0]
    print(f"Image ID:{image_id}")
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    
    print(f"Mask shape for Image ID {image_id}: {mask.shape}")
    visualize.display_top_masks(image, mask, class_ids, dataset.class_names)


# 3. Inference and Visualization

## 3.1 Load Models

After loading our datasets, we will load in our trained models. For ease of use, multiple models can be loaded by adding their path to the `model_paths` list. In this example, we are using our previously trained SAGE and COCO models, located in the `pretrained_models` directory.

Model Information:
* `SAGE_0`: This model is trained using solely synthetically generated TEM images and their corresponding masks (initialized with COCO Weights). 
* `SAGE_1`: Using the weights from `SAGE_0` for initialization, this model trains on a set of manually segmented real TEM images
* `SAGE_2`: Initialized with `SAGE_1` weights, this model trains again on the same real TEM images, but each image is now segmented by a different analyst than in `SAGE_1`.

* `COCO_1`: This model is trained in the same manner as `SAGE_1`, except it uses COCO weights for initialization rather than the previously trained synthetic-image-based `SAGE_0` model.
* `COCO_2`: Similar to `SAGE_2`, this model is initialized with weights from `COCO_1`, and trained on the same images, segmented by a different analyst than in `COCO_1`.


We also subclass our config to create an inference config that can be used to easily change batch size (`GPU_COUNT`*`IMAGES_PER_GPU`) and confidence thresholds (`DETECTION_MIN_CONFIDENCE`). The confidence threshold determines what confidence score is required for a model to view a prediction as "correct". For example, if we set a confidence threshold of 0.6, any predictions with a score of 0.6 or higher will be returned as possible match. 

After specifying model paths, we intialize a dictionary `model_dict` to store the models and relevant information.

Next, each model specified in `model_paths` is loaded and registered to the `model_dict`.

In [ ]:

#here, specify the model paths to be loaded

model_paths = [
    #os.path.join(PRETRAIN_DIR, "SAGE_0/SAGE_0.h5"),
    #os.path.join(PRETRAIN_DIR, "SAGE_1/SAGE_1.h5"),
    os.path.join(PRETRAIN_DIR, "SAGE_2/SAGE_2.h5"),
    #os.path.join(PRETRAIN_DIR, "COCO_1/COCO_1.h5"),
    os.path.join(PRETRAIN_DIR, "COCO_2/COCO_2.h5")
]

#subclass config for inference

class InferenceConfig(SAGEConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.6 #minimum confidence score for prediction to be accepted

inference_config = InferenceConfig()

#initialize dict and list for model storage/access
model_dict = {}
model_list = []

#load each model in "model_paths"
for path in model_paths:
    utils.load_model(path, model_dict, model_list, MODEL_DIR, inference_config)
    
    
#print list of loaded models    
utils.print_active_models(model_dict)    

## 3.2 Save Mask Overlays

This section provides the ability to save overlays of the masks created by both the model and the loaded dataset. 

### 3.2a Dataset Masks

We can view the list of loaded datasets using `utils.print_loaded_datasets()` to easily fetch the correct name keys of each dataset. 

To save the overlays of a dataset, select which dataset to save (`dataset_name`) and set boolean `save=True`. If the dataset contains manually segmented images (as opposed to some other imported masks from another segmentation method), set the boolean `Ground_Truth=True`.

If you are saving overlays of a dataset containing segmentation masks from imported masks from another segmentation method (to compare against the 'ground truth' manual masks), set `Ground_Truth=False` to ensure it saves within the proper directory. 

We can also toggle whether we want to view overlays with the boolean `view_overlay`.

In [ ]:
utils.print_loaded_datasets(datasets)

## Parameters for overlay viewing/saving
dataset_name = 'PROCI_Test'
dataset_analyze = datasets.get(dataset_name, None)
save = False
view_overlays = True
Ground_Truth = True

In [ ]:


if dataset_analyze and save:
    if not Ground_Truth:
        vis_dir = os.path.join(Results_DIR,"PROCI_Test", "Visualizations",dataset_name, "Display_instances")
    else:
        vis_dir = os.path.join(Results_DIR,dataset_name, "Visualizations", "Ground Truths")
    os.makedirs(vis_dir, exist_ok=True)

image_ids= dataset_analyze.image_ids
for image_id in image_ids:
    #print(f"Image ID: {image_id}")
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_analyze, inference_config, 
                               image_id)

    log("original_image", original_image)
    log("image_meta", image_meta)
    log("gt_class_id", gt_class_id)
    log("gt_mask", gt_mask)
      
    #print(ious_for_image)
    original_filename = dataset_analyze.image_info[image_id]['basename']
    base_filename = os.path.splitext(os.path.basename(original_filename))[0] 
    print(f"Image ID: {image_id}, File Path: {base_filename}")
    if save:    
        save_path = os.path.join(vis_dir, f"{base_filename}_visualization.png")
        print(f"Saving visualization to {save_path}")
    else:
        save_path=None
        print("saving not activated")
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                dataset_analyze.class_names, figsize=(8, 8),show_mask=False, show_bbox=False, 
                                    show_caption=False, linewidth = 2, save_path=save_path, view=view_overlays)


### 3.2b Model Prediction Masks


First, select which model to use for analysis (`model_name=`). We can view the list of loaded models using `utils.print_active_models()` to easily fetch the correct name keys of each model. 

Next, select which dataset to analyze (`dataset_analyze=`). This will provide the model with images to make predictions on.

Similar to saving the dataset masks, setting `save=True` will save images with the masks overlaid, as well as binary images of each prediction mask.

Next, choose whether you want to view each overlay with the `view_overlays` boolean.

In [ ]:
#utils.print_active_models(model_dict)

#Model overlay settings

model_name = 'SAGE_0'
dataset_name = 'PROCI_Test'
save = False
view_overlays = False


In [ ]:
## Save model predictions as overlays

#extract model and confidence threshold from model dict
model = model_dict[model_name]['model']
threshold = model_dict[model_name]['confidence']

#extract images to run model on from dataset
dataset_analyze = datasets.get(dataset_name, None)
image_ids= dataset_analyze.image_ids

#create visualization directory
vis_dir = os.path.join(Results_DIR, dataset_name, "Visualizations", f"{model_name}_{threshold}", "Display_instances")
os.makedirs(vis_dir, exist_ok=True)

#create directory to save individual masks
particle_dir = os.path.join(vis_dir, "particle")
os.makedirs(particle_dir, exist_ok=True)

print(f"Model {model_name}'s predictions above {threshold} confidence score on {dataset_name} images\\")

for image_id in image_ids:
    print(f"Image ID: {image_id}")
    image = dataset_analyze.load_image(image_id)
    mask, class_ids = dataset_analyze.load_mask(image_id)

    print(dataset_analyze.class_names)
    results = model.detect([image], verbose=1)

    r = results[0]
    
    original_filename = dataset_analyze.image_info[image_id]['basename']
    base_filename = os.path.splitext(os.path.basename(original_filename))[0] 
    base_num = base_filename.split('_')[-1]
    print(f"Image ID: {image_id}, File Path: {base_filename}")
    #print(r)
    if save == True:
        save_path = os.path.join(vis_dir, f"{base_filename}_predictions.png")
        print(f"Saving visualization to {save_path}, saving masks to {particle_dir}")
        
        #save particle masks
        masks = r['masks']
        
        for i in range(masks.shape[-1]):
            mask = masks[:,:,i].astype(np.uint8) #ensure binary?
            img = Image.fromarray(mask*255)
            fname = f"mask_{base_num}_{i:06d}.png"
            img.save(os.path.join(particle_dir,fname))


        
    else:
        save_path = None
        print("saving not activated")
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], dataset_analyze.class_names, scores=r['scores'],
                                show_caption=False,show_bbox=False, show_mask=True, linewidth=2, save_path=save_path, view=view_overlays)#, ax=get_ax())


## 3.3 IoU Heatmaps

IoU heatmaps provide a per-instance visualization of how well model predictions overlap with ground truth masks. In this section, we generate "IoU heatmaps", that color-code each prediction by its IoU score (green = high overlap/good prediction, red = low, pink = unmatched prediction). 

We start by selecting the **model/method name** (e.g., `SAGE_0` or `PROCI_EDMWSv2`), then the **reference dataset** that supplies the ground truth masks (`PROCI_Test`).

Then, we can specify certain settings to pass to the heatmap visualization:
* `metric_name` (str): metric to visualize e.g. (IoU)
* `normalize_metric` (bool): whether to normalize metric between 0 and 1 (True) or min and max (False)
* `show_caption` (bool): display captions or labels on predictions
* `show_bbox`(bool): display bounding box for predictions
* `show_mask` (bool): display predicted mask overlay (filled in)
* `show_cbar` (bool): display colorbar for metric values 
* `colormap` (str): color scheme for heatmapping (e.g., `'RdYlGn'` - check matplotlib for other options)
* `cbar_position`(list): locatiopn and size of colorbar `[x0, y0, width, height]`,
* `label_color` (str): color for text labels on masks.
* `show_pred_idx` (bool): display prediction index numbers
* `unmatched_color`(tuple): RGB color for predicitons that don't match any ground truth mask (e.g, pink)

The model is then run on the reference dataset, and the generated heatmaps are displayed and saved to the reference dataset's results directory



In [ ]:
utils.print_active_models(model_dict)

In [ ]:
method_name = 'SAGE_2'
ref_name = 'PROCI_Test'
model_dict =  model_dict
datasets= datasets
config = InferenceConfig

heatmap_settings = dict(
            metric_name="IoU",
            normalize_metric=False,
            show_caption=False,
            show_bbox=False,
            show_mask=True,
            show_cbar=False,
            colormap='RdYlGn',
            cbar_position=[0.1, 0.1, 0.8, 0.05],
            label_color='black',
            show_pred_idx=False,
            unmatched_color=(1.0, 105/255.0, 180/255.0)
            )



In [ ]:



filtered_df, full_name, model, dataset, ref_data = visualize.get_filtered_df(method_name,
                                                                   ref_name,
                                                                   model_dict, datasets,
                                                                   config,
                                                                   filter_size=False,
                                                                   verbose=False)
visualize.get_iou_heatmaps(filtered_df = filtered_df,
                ref_data = ref_data, model =model, 
                 dataset=dataset, method_name = full_name, 
                 config=config,
                 ref_name = ref_name, 
                 Results_DIR = Results_DIR, 
                 verbose = False, save_images=False, vis_settings=heatmap_settings)

# 4. Evaluation

The evaluation section of this notebook will allow us to compare the performance of multiple models or loaded segmentation methods through various methods, as well as save performance metrics for later viewing and comparison

## 4.1 Comparing IoU Distributions of multiple models

The quality of a model's predictions can be quantified by their Intersection-over-Union (IoU) score. In this section, we compare IoU's by plotting their reverse cumulative IoU distributions against a chosen reference dataset (`PROCI_Test`).
* `methods`: list of models or methods (imported datasets) to include in comparison using the name key of loaded models or datasets (e.g., `SAGE_0, SAGE_1, SAGE_2`)
* `ref_dataset`: dataset used as the "ground truth" for evaluation
* `method_styles`: optional dictionary to customize labels and colors for each method in the plot

The function `visualize.plot_rev_cum_iou()` generates the plots, allowing us to visually assess how well a model/method overlaps with ground truth masks across the full IoU range. Results are saved in the `IoU Distributions` folder for the chosen reference dataset.

Some key options for `visualize.plot_rev_cum_iou()`:
* `iou_threshold` (float, default=0): IoU cutoff threshold for processing matches
* `iou_summary` (bool, default=False): whether to print an IoU statistics summary for each model passed
* `fill` (bool, default=True): whether to fill the area under the curve
* `method_styles` (dict or None, default=None): optional custom style settings (label, color) for each method plotted
* `title` (bool, default=False): whether to show title on plot
* `verbose` (bool, default=False): print additional debug and tracking information


In [ ]:
#utils.print_active_models(model_dict)
#utils.print_loaded_datasets(datasets)

#names of models or datasets you want to compare
methods = [ 'SAGE_2', 'COCO_2', 'PROCI_EDMWSv2']

#dataset to test models/methods against
ref_dataset = 'PROCI_Test'

#optional dictionary to define plotting styles for models/methods
method_styles = {
    'SAGE_2': {
        'label': r'SAGE$_2$',
        'color': '#2ca02c' #green
    },
     'COCO_2':{
         'label': r'COCO$_2$',
        'color': '#ff7f0e' #orange
     },
    'PROCI_EDMWS':{
        'label':'EDM-WS (Full Masks))',
        'color':'#1f77b4' #blue
    },
    'SAGE_1':{
        'label':'SAGE_1',
        'color': '#d62728' #red
    }
}


In [ ]:
                   
#save_dir = os.path.join(Results_DIR, ref_dataset,"Visualizations", "IoU Distributions")
save_dir=None
visualize.plot_rev_cum_iou(methods,
                           model_dict=model_dict,
                           dataset_dict=datasets,
                           ref_dataset=ref_dataset,
                           config=inference_config,
                           save_dir=save_dir,
                           iou_threshold=0,
                           iou_summary=False,
                           method_styles=method_styles,
                           fill=True,
                           title=True,
                           verbose=False)
                           

## 4.2 Postprocessing/Results

Here, we will run a selected model (`model_name`) on the desired ground truth dataset (`ref_dataset`) to compile various morphological information and performance metrics, and save them into our results directory 

(TODO verify/add view only option)

After selecting our model and ground truth dataset, we create a `summary_settings` dict to control some overall functions in each postprocessing loop:
* `dataset_name` (str): Name of ground truth dataset
* `model_name` (str): Name of model to run
* `datasets` (dict): previously loaded datasets dictionary 
* `model_dict` (dict): previously loaded model dictionary
* `Results_DIR` (str): path to save metrics (usually predefined)
* `verbose` (bool): verbosity flag to print additional information/debug
* `save_results` (bool): saves results to Results_DIR

Next we will set the scale for our images (nm/pixel). If we are using an image set containing images of different scales, create a `scales` dictionary, and assign the correct scale to each image. If all images in a set have the same scale, we can just set `scales` as a single float value. 

The postprocessing/results gathering portion is broken into three main functions:
* 1. `gather_aggregate_morphology()` is a wrapper function that takes each image and determines various morphological information about the aggregate and primary particles in an image. This will run images through a pipeline to determine morphological information such as # of primary particles, mean primary particle diameter, radius of gyration, and fractal dimension of aggregates, returning a dataframe summarizing aggregate information (`aggregate_summary`) as well as a dataframe containing individual primary particle information (`pp_info`) a .csv file in the results directory. If saving is enabled in `summary_settings`, it will also save these into .csv files in the results directory for the corresponding reference dataset and model. This function is useful for analyzing datasets with or without ground truth particle masks, as it returns morphology information depending on the provided predictions, not ground truth labels. 

* 2. `calc_performance_metrics()` will run the model on the reference dataset, returning key machine learning performance metrics, such as confusion matrix values (TP,FP, FN), Accuracy, F<sub>1</sub> score, Average Precisions, and mean IoU. Enabling `save_results` in the summary settings will save these metrics to a csv file containing the metrics for all models/methods run on a reference dataset. This function is useful when determing how accurate a model is against a ground truth dataset. If no ground truth information is available, there is no need to run this function. 

* 3. `full_summary()` will take the outputs of `gather_aggregate_morphology()` and `calc_performance_metrics()`, returning (and saving) a full summary of both performance and morphological information.  
 
 


In [ ]:
#Choose Model/method
model_name = 'COCO_2' #(str or None)
dataset_name = 'PROCI_Test' #(str or None) - use this if analyzing dataset of imported method
ref_name = 'PROCI_Test'#'NS40_test' #why do I have both ref_name and dataset_name?


#settings for postprocessing/results gathering
summary_settings = dict(dataset_name = ref_name, #dataset name
                        model_name = model_name, #model name
                        datasets = datasets, #loaded datasets dict
                        model_dict=model_dict, # dict of models
                        Results_DIR = Results_DIR, #results directory
                        verbose = 1, #verbose toggle
                        save_results = False) #saving toggle
    



#TODO - check if single scale can be passed instead of dict
#TODO - double check unit


scales = {
    'image_000007':  0.2847, #scale of kunfeng images
    'image_000010': 0.16775516, #scale of dreier images
    'image_000017': 0.2847,
    'image_000020': 0.16775516,
    'image_000024': 0.16775516,
    'image_000028': 0.2847,
    'image_000043': 0.2847,
    'image_000049': 0.2847,
    'image_000053': 0.2847,
    
}

#or
#scales = 0.2847 float value if all images have same scale 

aggregate_summary, pp_info = utils.gather_aggregate_morphology(ref_name,
                                                               scales,
                                                               summary_settings,
                                                               save_binary=False,
                                                               show_binary=False, 
                                                               plot=False)

metrics = utils.calc_performance_metrics(ref_name, 
                                         inference_config,
                                         summary_settings,
                                         sort_method='confidence')
summary = utils.full_summary(aggregate_summary, metrics, pp_info, summary_settings)